In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Conv1D,Add,Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import cv2
import pandas as pd
from tqdm import tqdm
import os
import natsort
from glob import glob
import numpy as np
import warnings
import json

warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(physical_devices, 'GPU')

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    #! GPU 사용 가능
    for gpu in gpus:
        print("GPU:", gpu)
else:
    #! GPU 사용 불가능
    print("GPU not found")

### Data load 

In [ ]:
image_size=24

path=f"../data/modeling_data/train/"

height_train=np.load(path+'Height_train.npy')
ndvi_train=np.load(path+'NDVI_train.npy')
slope_train=np.load(path+'Slope_train.npy')
landuse_train=np.load(path+'Landuse_train.npy')
popden_train=np.load(path+'population_density_train.npy')

tabular_data=pd.read_csv(path+"climate_train.csv")
tabular_data.drop(['lon', 'lat'],axis=1,inplace=True)
tabular_data=tabular_data.replace(32767.0,-9999)
#tabular_data=tabular_data.replace(-9999.0,0)
tabular_data[['rainfall','temp','windspeed','humidity']].plot()
x,y=[],[]
for j in tqdm(range(len(tabular_data))):
    x.append(np.array(tabular_data.loc[j, ['humidity', 'rainfall', 'temp', 'windspeed']]).astype(float))
    y.append(np.array(tabular_data.loc[j, ['target']]).astype(float))
climate = np.array(x)
y = np.array(y)

In [ ]:
height_input = Input(shape=(image_size, image_size, 3), name='height_input')
ndvi_input = Input(shape=(image_size, image_size, 3), name='ndvi_input')
slope_input = Input(shape=(image_size, image_size, 3), name='slope_input')
landuse_input = Input(shape=(image_size, image_size, 3), name='landuse_input')
popden_input = Input(shape=(image_size, image_size, 3), name='popden_input')
climate_input = Input(shape=(4,1), name='climate_input')

def residual_block(input_layer, filters, kernel_size):
    x = Conv2D(filters, kernel_size, padding='same')(input_layer)
    x = Conv2D(filters, kernel_size, padding='same')(x)
    
    if input_layer.shape[-1] != filters:
        input_layer = Conv2D(filters, (1, 1), padding='same')(input_layer)
    
    x = Add()([input_layer, x])
    x = Activation('relu')(x)
    return x

def res_layers(input_layer):
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(input_layer)
    x = MaxPooling2D((2, 2), strides=(2, 2), padding='same')(x)
    
    x = residual_block(x, 64, (3, 3))
    x = residual_block(x, 64, (3, 3))
    
    x = residual_block(x, 128, (3, 3))
    x = residual_block(x, 128, (3, 3))
    
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    
    return x

def cnn_layers(input_layer):
    conv1_im = Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(input_layer)
    pool1_im = MaxPooling2D(pool_size=(2,2))(conv1_im)
    
    conv2 = Conv2D(64, kernel_size=(3,3), activation='relu', padding='same')(pool1_im)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 = Conv2D(128, kernel_size=(3,3), activation='relu', padding='same')(pool2)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    
    conv4 = Conv2D(256, kernel_size=(3,3), activation='relu', padding='same')(pool3)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    
    flatten = Flatten()(pool4)
    
    dense1 = Dense(128, activation='relu')(flatten)
    
    return dense1


def dense_layer(input_layer):
    dense0 = Dense(64, activation='relu')(input_layer)
    dense1 = Dense(64, activation='relu')(dense0)
    flatten = Flatten()(dense1)
    dense2 = Dense(64, activation='relu')(flatten)
    return dense2
    
height_cnn = cnn_layers(height_input)
ndvi_cnn = cnn_layers(ndvi_input)
slope_cnn = cnn_layers(slope_input)
landuse_cnn = cnn_layers(landuse_input)
popden_cnn = cnn_layers(popden_input)
climate_cnn = dense_layer(climate_input)

merged = concatenate([height_cnn, ndvi_cnn, slope_cnn, landuse_cnn, popden_cnn, climate_cnn])

output_layer = Dense(1, activation='sigmoid', name='output_layer')(merged)

model = Model(inputs=[height_input, ndvi_input,slope_input,landuse_input,popden_input,climate_input], outputs=output_layer)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
x_train = {
    'height_input': height_train,
    'ndvi_input': ndvi_train,
    'slope_input': slope_train,
    'landuse_input': landuse_train,
    'popden_input': popden_train,
    'climate_input':climate
}

### Mobile_net model (imagesize=32일때만 적용 가능 )

In [ ]:
# from tensorflow.keras.applications import MobileNet
# from tensorflow.keras.layers import Input, Dense, Flatten, concatenate
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # 입력층 정의
# height_input = Input(shape=(32, 32, 3), name='height_input')
# ndvi_input = Input(shape=(32, 32, 3), name='ndvi_input')
# slope_input = Input(shape=(32, 32, 3), name='slope_input')
# landuse_input = Input(shape=(32, 32, 3), name='landuse_input')
# popden_input = Input(shape=(32, 32, 3), name='popden_input')
# climate_input = Input(shape=(4, 1), name='climate_input')

# # MobileNet 모델의 인스턴스 생성
# mobilenet = MobileNet(include_top=False, weights='imagenet')

# def dense_layer(input_layer):
#     dense0 = Dense(64, activation='relu')(input_layer)
#     dense1 = Dense(64, activation='relu')(dense0)
#     flatten = Flatten()(dense1)
#     dense2 = Dense(64, activation='relu')(flatten)
#     return dense2

# # 입력층을 모델에 적용하여 특성 추출
# height_features = mobilenet(height_input)
# ndvi_features = mobilenet(ndvi_input)
# slope_features = mobilenet(slope_input)
# landuse_features = mobilenet(landuse_input)
# popden_features = mobilenet(popden_input)
# climate_features=dense_layer(climate_input)


# # 특성 맵을 일렬로 펼침
# flatten_height = Flatten()(height_features)
# flatten_ndvi = Flatten()(ndvi_features)
# flatten_slope = Flatten()(slope_features)
# flatten_landuse = Flatten()(landuse_features)
# flatten_popden = Flatten()(popden_features)
# flatten_climate = Flatten()(climate_features)

# # 특성 맵을 병합
# merged = concatenate([flatten_height, flatten_ndvi, flatten_slope, flatten_landuse, flatten_popden, flatten_climate])

# # 출력층 정의
# output_layer = Dense(1, activation='sigmoid', name='output_layer')(merged)

# # 모델 정의
# model = Model(inputs=[height_input, ndvi_input, slope_input, landuse_input, popden_input, climate_input], outputs=output_layer)

# # 모델 컴파일
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy',metrics=['accuracy'])

# model.summary()


In [ ]:
class AccuracyThresholdCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.95:
            print("Reached accuracy threshold (0.9). Stopping training.")
            self.model.stop_training = True
model.fit(x_train, y, epochs=100, batch_size=128, callbacks=[AccuracyThresholdCallback()])

In [ ]:
model.save(f"C:/Users/user/fire_model/cnn24") 